# The Convex Hull Problem

Pound a bunch of nails into a board, then stretch a rubber band around them and let the rubber band snap taut, like this:

<img src="http://www.personal.kent.edu/~rmuhamma/Compgeometry/MyCG/Gifs-CompGeometry/ch2.gif">

The rubber band has traced out the *convex hull* of the set of nails. It turns out this is an important problem with applications in computer graphics, robot motion planning, geographical information systems, ethology, and other areas.
More formally, we say that:

*Given a finite set, **P**, of points in a plane, the convex hull of **P** is a polygon, **H**, such that:*

- *Every point in **P** lies either on or inside of **H**.*
- *Every vertex of **H** is a point in **P**.*
- **H** *is convex: a line segment joining any two vertexes of **H** either is an edge of **H** or lies inside **H**.*


In this notebook we develop an algorithm to find the convex hull (and show examples of how to use `matplotlib` plotting). The first thing to do is decide how we will represent the objects of interest:

- **Point**: We'll define a class such that `Point(3, 4)` is a point where `p.x` is 3 and `p.y` is 4.
- **Set of Points**: We'll use a Python set: `{Point(0,0), Point(3,4), ...}`
- **Polygon**: We'll represent a polygon as an ordered list of vertex points.

First, get the necessary imports done:

In [2]:
from __future__ import division, print_function

%matplotlib inline 
import matplotlib.pyplot as plt
import collections
import random
import math

# Points and Sets of Points

I'll define the class `Point` as a named tuple of `x` and `y` coordinates, and `Points(n)` as a function that creates a set of *n* random points. 

There are two complications to the function `Points(n)`:
1. A second optional argument is used to set the random seed.  This way, the same call to `Points` will return the same result each time.  That makes it easier to reproduce tests.  If you want different sets of points, just pass in different values for the seed.
2. Since `matplotlib` plots on a 3&times;2 rectangle by default, the points will be uniformly sampled   from a 3&times;2 box (with a small border of 0.05 on each edge to prevent the points from bumping up against the edge of the box).

In [2]:
Point = collections.namedtuple('Point', 'x, y')

def Points(n, seed=42):
    "Generate n random points within a 3 x 2 box."
    random.seed((n, seed))
    b = 0.05 # border
    return {Point(random.uniform(b, 3-b), random.uniform(b, 2-b)) 
            for _ in range(n)}

In [3]:
Points(3)

{Point(x=0.15172583449638682, y=1.6108693392839208),
 Point(x=0.968326330695687, y=1.3139550880088586),
 Point(x=1.3508070075242857, y=0.22290610532132638)}

# Visualizing Points and Line Segments


Now let's see how to visualize points; I'll define a function `plot_points`.  We will want to be able to see:
- The **points** themselves. 
- Optionally, **line segments** between points. An optional `style` parameter allows you to specify whether you want lines or not, and what color they should be. This parameter uses the standard [style format](http://matplotlib.org/1.3.1/api/pyplot_api.html#matplotlib.pyplot.plot) defined by matplotlib; for example, `'r.'` means red colored dots with no lines, `'bs-'` means blue colored squares with lines between them, and `'go:'` means green colored circles with dotted lines between them.  The lines go from point to point in order; if you want the lines to close
back from the last point to the first (to form a complete polygon), specify `closed=True`. (For that to work,
the collection of points must be a list; with `closed=False` the collection can be any collection.)
- Optionally, **labels** on the points that let us distinguish one from another. You get
labels (integers from 0 to *n*) if you specify `labels=True`.

In [4]:
def plot_points(points, style='r.', labels=False, closed=False): 
    """Plot a collection of points. Optionally change the line style, label points with numbers, 
    and/or form a closed polygon by closing the line from the last point to the first."""
    if labels:
        for (i, (x, y)) in enumerate(points):
            plt.text(x, y, '  '+str(i))
    if closed:
        points = points + [points[0]]
    plt.plot([p.x for p in points], [p.y for p in points], style, linewidth=2.5)
    plt.axis('scaled'); plt.axis('off')